#### Задание 1.
Построить алгоритм и проргамму для вычисления двукратного интеграла
$$
I = \iint_G f(x, y) dx dy
$$
по области $G$, ограниченной прямыми
$$
x + y = 1, x = 0, y = 0.
$$

Функция $z = f(x, y)$ задана таблично в файле `data.csv`.

Применить метод последовательного интегрирования. По одному направлению использоавть формулу Гаусса, а по другому - формулу Симпсона.

*Указание.* При двумерной интерполяции по таблице функции $z = f(x, y)$ использовать выравнивающие переменные $(\eta, x, y)$, причём $\eta = \ln z$.

**Результаты.**
1. Разработать алгоритм вычисления $n$ корней полинома Лежандра $n$-ой степени $P_n(x)$ при реализации формулы Гаусса.
2. Исследовать влияние количества выбираемых узлов сетки по каждому направлению на результаты расчётов.

In [2]:
import numpy as np
import sympy as sp
import pandas as pd

# Read table
df = pd.read_csv('data.csv')

x = np.linspace(0, 1.2, 13, endpoint=True)
y = np.linspace(0, 1.2, 13, endpoint=True)
z = df.values

# Alignment variable
eta = np.array([[np.log(i) for i in zi] for zi in z])

def f(x):
    return x ** 2

def get_quadrature(k):
    return 2 / (k + 1) if k % 2 == 0 else 0

# def get_legendre_polynome(n):
#     x_symb = sp.Symbol('x')
#     n_symb = sp.Symbol('n')
#     lhs = 1 / (2**n_symb * sp.factorial(n_symb))
#     rhs = (x_symb**2 - 1)**n_symb
#     lhs = lhs.subs({n_symb: n})
#     rhs = rhs.subs({n_symb: n})
#     lp = lhs * sp.diff(rhs, x_symb, n)
#     legendre_polynome_fn = sp.lambdify(x_symb, lp)
#     return legendre_polynome_fn

from numpy.polynomial.legendre import Legendre
def get_legendre_polynomial_roots(n):
    return Legendre(n * [0] + [1]).roots()

def get_coefficients(legendre_polynomial_roots):
    roots = legendre_polynomial_roots
    n = len(roots)
    
    matrix = np.array([[t ** t_power for t in roots] for t_power in range(n)])

    quadrature_values = np.array([get_quadrature(k) for k in range(n)])
    
    coefs = np.linalg.solve(matrix, quadrature_values)
    
    return coefs
    
def integrate_gauss(func, a, b, n=100):
    roots = get_legendre_polynomial_roots(n)
    coefs = get_coefficients(roots)
    
    xs = np.array(list(map(lambda t: (b - a) / 2 * t + (a + b) / 2, roots)))
    
    result = (b - a) / 2 * sum(coefs * func(xs))

    return result

def integrate_simpson(func, a, b, n=100):
    h = (b - a) / (n*2)
    
    xs = np.array([a + i * h for i in range(n*2 + 1)])
    ys = np.array([func(x) for x in xs])
    
    s1 = 4 * np.sum([ys[2*i - 1] for i in range(1, n)])
    s2 = 2 * np.sum([ys[2*i] for i in range(1, n - 1)])
    s = (h / 3) * (ys[0] + s1 + s2 + ys[-1])
    
    return s

integrate_gauss(f, 2, 5)
# integrate_simpson(f, 2, 5)

38.99999999999997

#### Задание 2.

Задана табличная (сеточная) функция. Имеется информация, что закономерность, представленная этой таблицей, может быть описана формулой
$$
y = \frac{a_0 x}{a_1 + a_2 x},
$$
параметры функции неизвестны **и определять их не нужно**.

| x | y | 1 | 2 | 3 | 4 | 5 |
| - | - | - | - | - | - | - |
| 1 | 0.571 |   |   |   |   |   |
| 2 | 0.889 |   |   |   |   |   |
| 3 | 1.091 |   |   |   |   |   |
| 4 | 1.231 |   |   |   |   |   |
| 5 | 1.333 |   |   |   |   |   |
| 6 | 1.412 |   |   |   |   |   |

Вычислить первые разностные производные от функции и занести их в стоблцы (1)-(4) таблицы:
* 1 - односторонняя разностная производная,
* 2 - центральная разностная производная,
* 3 - 2-я формула Рунге с использованием односторонней производной,
* 4 - введены выравнивающие переменные
* 5 - вторая разностная производная

**Результаты.**
Заполненная таблица с краткими комментариями по поводу используемых формул и их точности.